In [45]:
import ollama
import pandas as pd
import json
import re

In [46]:
MODEL = "qwen3:32b"

In [47]:
# load agent2_ideas.json file
with open("output/agent2_ideas.json", "r") as f:
    data = json.load(f)

# convert to pandas DataFrame
df = pd.DataFrame(data)
df.columns

Index(['title', 'description', 'data_needs', 'idea_id'], dtype='object')

In [48]:
prompt1 = f"""
You are an AI Feasibility Analyst. Your task is to meticulously evaluate the practical viability of a proposed research idea related to <Topic>Climate Change and Deep Learning</Topic>. You will be provided with the idea's title, description, and data requirements.

Based on this information, you must:
1.  **Evaluate Feasibility Dimensions:** Thoroughly assess the idea against the following dimensions, making internal notes on each:
    *   **Data Volume & Availability:** Consider the scale, accessibility, and preparation challenges of the required data. Note any significant hurdles (e.g., "requires petabytes of proprietary data," "extensive manual labeling needed") or facilitators (e.g., "uses readily available public datasets").
    *   **Model Fit & Complexity:** Evaluate the suitability of deep learning. Note if standard models apply or if it requires novel, highly complex, or exceptionally resource-intensive models. Consider the technical risk associated with the proposed modeling approach.
    *   **Compute Needs:** Estimate the computational resources. Note if it implies standard GPU usage, high-performance computing, or specialized hardware, and whether this is a significant barrier.
    *   **Deployment Path & Scalability:** Assess the complexity of real-world implementation if successful. Note potential challenges in scalability, maintenance, or integration.

2.  **Synthesize into a Feasibility Score:** Based on your detailed assessment of the dimensions above, assign a single numerical Feasibility Score.
    *   The score MUST be a value between 0.0 and 1.0, inclusive.
    *   0.0 indicates extreme unfeasibility; 1.0 indicates high feasibility.

3.  **Formulate Justification Notes:** Based on your evaluation of the dimensions, compose concise notes that clearly justify the assigned Feasibility Score. These notes should highlight the primary factors (both positive and negative) that influenced your scoring across the different feasibility dimensions. Aim for 2-4 key points in your notes.

4.  **Provide Output in Specified Format:** Your response must contain both the Feasibility Score and the Notes.

**Input Research Idea Details:**
---
"""
promp2 = f"""
**Output Requirement:**

Your response MUST be formatted *strictly* as follows, with no other text, explanations, or characters outside this structure on a single line:
`Feasibility Score: x, Notes: [Your concise notes explaining the score, highlighting key factors from data, model, compute, and deployment considerations]`

**Example of desired output format:**
`Feasibility Score: 0.65, Notes: Moderate data acquisition challenges due to specificity, but utilizes established model architectures. Compute needs are manageable with standard GPUs. Deployment path requires integration with existing sensor networks which could be complex.`

OR

`Feasibility Score: 0.3, Notes: Relies on extremely large, currently unavailable datasets. Proposed model is highly experimental with significant technical risk. Deployment would require substantial new infrastructure.`

Now, analyze the provided idea details and output the Feasibility Score and corresponding Notes in the specified format.

"""

In [49]:
def query_ollama_model(model_name, PROMPT_FOR_KEYWORDS):
    """
    Sends a prompt to the specified Ollama model and returns the response.

    Args:
        model_name (str): The name of the Ollama model to use (e.g., "llama3.2:3b").
        prompt_text (str): The prompt to send to the model.

    Returns:
        str: The content of the model's response, or an error message.
    """
    try:
        print(f"Sending prompt to Ollama model: {model_name}...")
        response = ollama.chat(
            model=model_name,
            messages=[
                {
                    'role': 'user',
                    'content': PROMPT_FOR_KEYWORDS,
                },
            ]
        )
        # print("Received response from Ollama.")
        return response['message']['content']
    except Exception as e:
        return f"An error occurred while communicating with Ollama: {e}"

def get_ollama_response(PROMPT_FOR_KEYWORDS,OLLAMA_MODEL_NAME=MODEL):

    # --- Execution ---
    # print(f"Requesting keywords from Ollama for model: {OLLAMA_MODEL_NAME}...")
    
    keywords_response = query_ollama_model(OLLAMA_MODEL_NAME, PROMPT_FOR_KEYWORDS)
    return keywords_response

In [50]:
def extract_feasibility_details(ollama_response_string):
    """
    Extracts the feasibility score (as a float) and notes from the
    Ollama model's response string.
    """
    # 1. clean up any markdown/backticks
    cleaned = ollama_response_string.strip(' \t\n\r`*')

    # 2. look for "Feasibility Score" plus a number (with optional "might be around")
    score_pattern = re.compile(
        r"feasibility\s+score\s*(?:[:=]|\s+might\s+be\s+around)?\s*"
        r"(?P<score>\d+(?:\.\d+)?)",
        re.IGNORECASE
    )
    score_match = score_pattern.search(cleaned)
    if not score_match:
        print("Error: no feasibility score found in response.")
        return None, None

    # 3. parse the score
    try:
        feasibility_score = float(score_match.group("score"))
    except ValueError:
        print(f"Error: could not convert '{score_match.group('score')}' to float.")
        return None, None

    # 4. now try to extract a literal "Notes:" block
    notes_pattern = re.compile(r"notes\s*[:=]\s*(?P<notes>.+)", re.IGNORECASE)
    notes_match = notes_pattern.search(cleaned)

    if notes_match:
        notes = notes_match.group("notes").strip()
    else:
        # fallback: everything after the score match
        notes = cleaned[score_match.end():].lstrip(' .,:-').strip()

    return feasibility_score, notes


In [51]:
for idx, row in df.iterrows():
    # get the value of the "ideas" column
    title = row["title"]
    description = row["description"]
    data_needed = row["data_needs"]
    idea_id = row["idea_id"]
    temp_prompt = f"""
    *   **Title:** {title}
    *   **Description:** {description}
    *   **Data Needed:** {data_needed}
    """
    
    prompt = prompt1 + temp_prompt + promp2
    feasibility_score = get_ollama_response(prompt)
    feasibility_score, notes = extract_feasibility_details(feasibility_score)
    #store the results in the DataFrame
    df.at[idx, "feasibility_score"] = feasibility_score
    df.at[idx, "feasibility_notes"] = notes

Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...
Sending prompt to Ollama model: qwen3:32b...


In [52]:
# keep only rows where title is nonempty
df = df[df['title'].str.strip().astype(bool)]

# (optional) reset the index to go 0…N-1 again
df = df.reset_index(drop=True)

In [53]:
df

,title,description,data_needs,idea_id,feasibility_score,feasibility_notes
0,Arctic Anomaly Detection with Computer Vision,A deep learning system that detects and classi...,"[Satellite imagery, Arctic climate data, histo...",8_arctic_anomaly_detection_with_computer_vision,0.65,Satellite imagery and climate data are partial...
1,Climate Resilience Prediction using GANs,Generative adversarial networks (GANs) simulat...,"[Historical climate data, urban/agricultural d...",9_climate_resilience_prediction_using_gans,0.70,Data availability is mixed with public climate...
2,Sea Level Rise Prediction with Deep Learning,A deep learning model that forecasts regional ...,"[Ocean current measurements, satellite altimet...",10_sea_level_rise_prediction_with_deep_learning,0.70,Public datasets for ocean and satellite data r...
3,Renewable Energy Output Forecasting,Machine learning algorithms predict solar and ...,"[Weather forecasts, satellite imagery, energy ...",11_renewable_energy_output_forecasting,0.75,Public weather and satellite datasets are wide...
4,Climate Policy Optimization with Reinforcement...,Reinforcement learning models simulate and rec...,"[Climate policy datasets, economic indicators,...",12_climate_policy_optimization_with_reinforcem...,0.65,Data integration challenges exist due to diver...
5,Sustainable Deep Learning Models for Energy Ef...,Designs energy-efficient deep learning archite...,"[Model performance metrics, energy consumption...",13_sustainable_deep_learning_models_for_energy...,0.85,Utilizes established lightweight model techniq...
6,Climate Data Fusion for Enhanced Predictions,A multimodal deep learning framework that fuse...,"[Satellite data, ground sensor networks, ocean...",14_climate_data_fusion_for_enhanced_predictions,0.70,Data integration challenges due to multimodal ...
7,AI-Driven Carbon Footprint Analysis,A deep learning system that tracks and predict...,"[Emissions inventories, supply chain data, ene...",15_ai-driven_carbon_footprint_analysis,0.65,Data availability is mixed (public emissions d...
8,Geoengineering Impact Assessment with Simulations,Machine learning models simulate the environme...,"[Atmospheric chemistry data, climate simulatio...",16_geoengineering_impact_assessment_with_simul...,0.60,Data challenges arise from limited historical ...
9,Climate Tipping Points Prediction with Predict...,Predictive analytics models identify early war...,"[Historical climate records, tipping point ind...",17_climate_tipping_points_prediction_with_pred...,0.60,Data availability is partially constrained by ...


In [54]:
# Still save the CSV with all data if needed
df.to_csv("output/ask4.csv", index=False)

# Save all ideas in one JSON file as an array of objects
# Convert the DataFrame to a list of dictionaries
ideas_list = df.to_dict(orient="records")

# Save all ideas to a single JSON file
with open("output/task4.json", "w") as f:
    json.dump(ideas_list, f, indent=4)

print(f"Saved {len(ideas_list)} ideas to output/task4.json")

Saved 20 ideas to output/task4.json
